In [1]:
import os
import pandas as pd
import seaborn as sns
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import scipy.stats as stats
import sklearn.manifold as skmf
import sklearn.decomposition as skd
import pickle
import lifelines
from sklearn.utils import resample
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score
from matplotlib import pyplot
from lifelines import CoxPHFitter
from lifelines.statistics import logrank_test
from lifelines.utils import median_survival_times
from lifelines.utils import k_fold_cross_validation
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import confusion_matrix

In [ ]:
# import original train data 
path= "C:/PSMAradiomics_export/analysisforshortpaper/"
file="train_original.csv"
full=path+file
dfLF=pd.read_csv(full,index_col=0)
dfLF.head()

In [5]:
dfLF[dfLF.index.duplicated()]
dfLF = dfLF[~dfLF.index.duplicated()]

In [ ]:
# apply SMOTE to original train data
x=dfLF.loc[:,'ft_last':'survtime']
y=dfLF[["survevent"]]
from imblearn.over_sampling import SMOTE
oversample= SMOTE()
X,Y=oversample.fit_resample(x,y)
print(Y)

In [9]:
# create dataframes of the new X and new Y
df_newX = pd.DataFrame(X)
df_newY= pd.DataFrame (Y)

In [11]:
# concatenate above dataframes horizontally
df_trainimbalancecorrected= pd.concat([df_newX, df_newY], axis=1)
# save to excel file
export_csv=df_trainimbalancecorrected.to_csv(r'C:/PSMAradiomics_export/analysisforshortpaper/train_imbcorrected.csv',index=False,header=True)

In [ ]:
# import traindata_imbalance corrected
path="C:/PSMAradiomics_export/analysisforshortpaper/
file="train_imbcorrected.csv"
full=path+file
df_trainimbalancecorrected=pd.read_csv(full,index_col=0)

In [5]:
df_trainimbalancecorrected[df_trainimbalancecorrected.index.duplicated()]
df_trainimbalancecorrected = df_trainimbalancecorrected[~df_trainimbalancecorrected.index.duplicated()]

In [7]:
X_train=df_trainimbalancecorrected[[:,'ft_last']]
y_train=df_trainimbalancecorrected[["survtime","survevent"]]

In [ ]:
# import original test data
path="C:/PSMAradiomics_export/analysisforshortpaper/"
file="testdata_original.csv"
full=path+file
df_test=pd.read_csv(full,index_col=0)
df_test.head()

In [ ]:
X_test=df_test[[:,'ft_last']]
y_test=df_test[["surtime","survevent"]]

In [17]:
# Fit the Cox model on the training data
cph = CoxPHFitter()
cph.fit(pd.concat([X_train, y_train], axis=1), duration_col='survtime', event_col='survevent')

# Predict on the test data
predictions = cph.predict_survival_function(X_test)

In [ ]:
#Evaluate test model using c-index
from lifelines.utils import concordance_index
c_index=concordance_index(y_test['survtime'], -predictions, y_test['survevent'])
print("C-index:", c_index)

In [ ]:
# For supervised feature selection, Model 1 used the top 12 features ranked by Mean Decrease Accuracy (MDA) from clinical, ring, and weighted GTVp/GTVn radiomics; 
# supervised ft selection- Model 2 used the top 12 MDA-ranked features from clinical and primary tumor, ring radiomics.
# For unsupervised selection, Model 1 used the top 12 highest-variance features from clinical, ring, and weighted GTVp/GTVn radiomics; 
# unsupervised ft selection- Model 2 used the top 12 highest-variance features from clinical and primary tumor, ring radiomics.
# Model 3 in all the feature set dimensionality reduction cases was built using clinical variables only. 

# Clinical variables- age, Gleason score, iPSA, PSA_relapse
# ft_last- is the last predictor in the list of predictors specified in the feature file (so basically selects the entire predictor list)
# survevent and survtime- MFS event (binary) and time to event data respectively
# highest-variance based feature selection code provided in R file in the GitHub folder